## Example: Tiling gridded data to XYZ structure and OSM structure

This example will show how one can export a raster dataset to individual tiles in one of the stuctures mentioned in the title above. Normally this can be done via the [DataCatalog](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.html) and getting gridded data via DataCatalog.[get_rasterdataset](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.get_rasterdataset.html). For this example we'll use our own created dummy data.

In [4]:
# Imports
from hydromt import raster
from numpy import random

# Create dummy data
gd = raster.full_from_transform(
        transform=[0.004166666666666666, 0.0, 0.0, 0.0, -0.004166666666666667, 0.0],
        shape=(21600, 21600),
        nodata=-9999,
        name="dummy_tile_data",
        crs=4326,
    )

# Add some values to the raster
gd.values = random.randint(-5000, 5000, (21600, 21600))

'gd' is gridded data as a xarray.DataArray object. With HydroMT a xarray.DataArray has some extra functionality via '.raster'.

This extra functionality does include the ability to write a raster to a tile database (tiling).

Let's first ensure there is a 'temp' folder to write the data to!


In [5]:
# For the examples, the data will be written to the temp folder on the C drive.
import os

folder = "C:\\temp"

if not os.path.exists(folder):
    os.mkdir(folder)

### XYZ stucture
First let's have a look at the XYZ structure.

A xarray.DataArray is simple written to a tile database in XYZ structure via .raster.to_xyz



In [6]:
# Write the database in XYZ stucture

gd.raster.to_xyz(
    root = f"{folder}\\dummy_xyz",
    px = 1024,
    zoomlevels = [0,1,3,4],
)

The database in the 'dummy_xyz' folder now contains 4 zoomlevels as defined above. 

zoomlevel 0 is at the scale of the xarray.DataArray (one on one). zoomlevel 1 is downscaled by a factor 2 compared to zoomlevel 0. zoomlevel 3 is downscaled by a factor of 8 compared to zoomlevel 0.

In the example above the amount of pixels per tile is determined at 1024 pixels by 1024 pixels. This resolution must NOT exceed the resolution of the total xarray.DataArray. In this example it should then be smaller than 21600.

A mosaic is created per zoomlevel of these tiles in a .vrt file.

At last, a .yml file is produced which can be read by the [DataCatalog](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.html) of HydroMT.

In [ ]:
# Imports
from hydromt import DataCatalog

# Load the yml into a DataCatalog
dc = DataCatalog(f"{folder}\\dummy_xyz.yml")

# View the structure of the DataCatalog
print(dc)

With DataCatalog.[get_rasterdataset](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.get_rasterdataset.html) a raster (.vrt) can be retreived. In case of a tile database it can be done for a certain zoomlevel. E.g.

In [ ]:
# Request a raster from the Datacatolog
rast = dc.get_rasterdataset("dummy_xyz", zoom_level=1)

# View the data
print(rast)

However, the tiles can also be individually used and read.

In [ ]:
# Imports
from hydromt import open_raster

# Chose a random tile from zoomlevel 1
rast = open_raster(f"{folder}\\dummy_xyz\\1\\5\\5.tif")

# View the data
print(rast)

### OSM structure
Second on our list is the OSM structure from [OpenStreetMap](https://www.openstreetmap.org).

A xarray.DataArray is simple written to a tile database in OSM structure via .raster.to_osm

In [ ]:
# Write the database in OSM stucture

gd.raster.to_osm(
    root = f"{folder}\\dummy_osm",
    zl = 5,
)

We won't go in depth here concerning the structure according to osm.
For this, visit: https://wiki.openstreetmap.org/wiki/Zoom_levels and subsequent references.

However, there are three things to be considered:

    - The resolution of a tile cannot be defined. This is predetermined at 256 pixels by 256 pixels.
    - CRS is always set at EPSG:3857
    - Zoomlevel 0 in the OSM structure spans the entire globe

In [ ]:
# Imports
from hydromt import DataCatalog

# Load the .yml in a DataCatalog
dc = DataCatalog(f"{folder}\\dummy_osm.yml")

# View the DataCatalog
print(dc)

Like the XYZ stucture, retrieve the gridded data via DataCatalog.[get_rasterdataset](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.get_rasterdataset.html).

In [ ]:
#Get a raster from the DataCatalog
rast = dc.get_rasterdataset("dummy_osm", zoom_level=3)

#View the data
print(rast)

Individual tiles:

In [ ]:
# Imports
from hydromt import open_raster

# Chose a random tile from zoomlevel 2
rast = open_raster(f"{folder}\\dummy_osm\\2\\2\\2.tif")

# View the data
print(rast)